# Pre-Download Data Processing

This notebook handles the preprocessing of YouTube watch history data before the video download phase. It performs several crucial data cleaning and filtering steps to prepare the dataset for the video sampling and download pipeline.

## Key Processing Steps:
1. Import watch histories from participant JSON files
2. Clean invalid or missing video IDs
3. Filter videos within specified date range (2019-2024)
4. Remove advertisements and YouTube Music entries
5. Export cleaned dataset for the download pipeline

## Requirements:
- Input: Watch history JSON files in participant-specific format
- Output: Cleaned CSV file ready for video sampling

## Dependencies:

In [ ]:
! pip install yt-dlp
! pip install scenedetect
! pip install opencv-python
! pip install ffmpeg

In [3]:
# import libraries
import pandas as pd
import yt_dlp as yt
import os
import numpy as np
import time
import random
import json
import zipfile
import sys
import ffmpeg
from datetime import datetime
from download_utils import *

sys.path.append('..')
from ytutils import *

## Sampling & Downloading

### Watch-Histories Import
- look into folder with watch-histories from Epinion in format [participant-id].json
- import watch-histories as dataframe

In [3]:
# Directory containg the JSON watch histories
wh_directory = '../Survey_Data/Import_Date_2024_12_02/Watch_Data'  #directory with .json files
path_wh = "watch_history.csv" #csv_path

if not os.path.exists(path_wh):
    print(loadEpinionData.__doc__)
    all_wh = loadEpinionData(wh_directory, save_dataframe=True)
else:
    all_wh = pd.read_csv(path_wh)


    This function creates one watch history dataframe from the inputted watch-history json files.
    --- args ---
    folder_path: string  # folder where watch-history files are located (.json)

    --- kwargs ---
    save_dataframe: bool  |  default: False

    --- output ---
    Outputs from function
    watch_history: pandas.DataFrame

    Outputs to current directory (if save_dataframe=True)
    watch_history: .csv
    
Processing file 1077/1077
Processing complete.


In [4]:
# Get the initial number of rows before filtering
initial_row_count = len(all_wh)

# Dropping rows where 'video_id' is NaN or empty string
all_wh = all_wh[all_wh['video_id'].notna() & (all_wh['video_id'].str.strip() != '')]

# Get the number of rows after filtering
final_row_count = len(all_wh)

# Ensuring Participant ID is regarded as str
all_wh['Participant ID'] = all_wh['Participant ID'].astype(str)

removed_n = initial_row_count - final_row_count
removed_pc = (removed_n / initial_row_count) * 100

# Print the number of dropped rows
print(f"Number of dropped rows (%): {removed_n} ({round(removed_pc, 2)}%).")

Number of dropped rows (%): 29164 (0.15%).


## Subsetting for recent years
- subset df for recent videos
    - start date: 2019-07-01
    - end date:   2024-06-30
- output:
    - output df
    - length of input df
    - length of output df

In [5]:
# Define the date range: from July 1, 2019 to June 30, 2024
start_date = '2019-07-01'  # '2019-07-01'
end_date = '2024-06-30'    # '2024-06-30'


# Subset the dataframe based on the date range
last5_wh = all_wh[(all_wh['time'] >= start_date) & (all_wh['time'] <= end_date)]

# Get the number of rows after subsetting
last5_row_count = len(last5_wh)

removed_n = final_row_count - last5_row_count
removed_pc = (removed_n / len(all_wh)) * 100

print(f"Number of videos watched outside the range from July 1, 2019 to June 30, 2024 (%): {removed_n} ({round(removed_pc, 2)}%).")

Number of videos watched outside the range from July 1, 2019 to June 30, 2024 (%): 6076450 (31.88%).


## Data Cleaning
- remove advertisements
- remove yt-music entries
- output:
   - clean df
   - metric on dropped ads in the new df
   - metric on dropped music videos in the new df

In [6]:
# Drop rows where the "details" column has a non-null entry (advertisement)
no_ads_wh = last5_wh[last5_wh['details'].isna()]

ads_n = len(last5_wh) - len(no_ads_wh)
ads_pc = (ads_n / len(last5_wh)) * 100
print(f"Number of dropped advertisement entries (%): {ads_n} ({round(ads_pc,2)}%).")


clean_wh = no_ads_wh[no_ads_wh["url"].apply(str).apply(len) <= 43]

music_n = len(no_ads_wh) - len(clean_wh)
music_pc = (music_n / len(last5_wh)) * 100
print(f"Number of dropped non-43 char URLs (YT Music) (%): {music_n} ({round(music_pc, 2)}%).")

Number of dropped advertisement entries (%): 2337818 (18.01%).
Number of dropped non-43 char URLs (YT Music) (%): 565916 (4.36%).


In [7]:
# save clean dataframe
clean_wh.to_csv('clean_watch_history.csv', index=False)

### Move to "Video_download_Pipeline.ipynb"